In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [11]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [12]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [13]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501902


In [14]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad
6017043,1112451244,8MPM5AO,0,20.0,True,False,False,False,False,False,...,False,False,False,False,Full-time,-,Consultoria,12.0,NaN,NaN
724493,1112477931,rm0mNBz,0,26.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Recursos Humanos,NaN,NaN,NaN
11470541,1112289570,xkP9vjK,1,42.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,12.0,NaN,NaN
9239103,1112512290,X9l3dWJ,0,26.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Marketing,32.0,NaN,NaN
12509081,1112187955,6rQ1xWj,0,32.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Transporte,NaN,NaN,NaN
834794,1112455114,dY5JLvb,1,39.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,421.0,2.0,0.004751
8951643,1112444304,A3XEArm,0,25.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,4.0,NaN,NaN
4878100,1112440565,8Ma4e4l,1,24.0,False,True,False,False,False,False,...,False,True,False,False,Full-time,Junior,Almacén / Depósito / Expedición,74.0,2.0,0.027027
6325229,1112451056,mzdN0b3,0,30.0,False,True,False,False,False,False,...,False,False,False,True,Part-time,Senior / Semi-Senior,Ventas,11.0,NaN,NaN
7740890,1112208794,LNelxdr,1,40.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,89.0,NaN,NaN


In [15]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
4789895,1112279973,6rQmAL4,0,26.0,False,True,False,False,False,False,...,False,False,True,Part-time,Senior / Semi-Senior,Atención al Cliente,0.0,0.0,0.000000,False
12073511,1111313442,ow2QpLr,0,200.0,False,False,True,False,False,False,...,True,False,False,Full-time,Junior,Contabilidad,63.0,0.0,0.000000,False
9921449,1112359198,A39NDYm,0,24.0,False,True,False,False,False,False,...,False,True,False,Full-time,Otro,Ventas,0.0,0.0,0.000000,False
6924728,1112371684,3NkL2XK,1,29.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ventas,139.0,0.0,0.000000,False
6691295,1112466925,GNo8pLN,1,24.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Abastecimiento,61.0,1.0,0.016393,True
999477,1112460147,mzPLqOY,1,24.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Administracion de Seguros,41.0,1.0,0.024390,True
4248459,1112458318,8w25ED,1,42.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Control de Gestión,11.0,0.0,0.000000,False
8726925,1111537091,xkdvZzP,0,24.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Seguridad,77.0,0.0,0.000000,False
10291862,1112409121,3NkD40r,0,25.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,55.0,0.0,0.000000,False
9323376,1112272731,owE4P5a,1,32.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Comercial,0.0,0.0,0.000000,False


In [16]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Almacén / Depósito / Expedición,1
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [17]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
2,0z5Dmrd,Junior,2
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,16


In [18]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,20
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,28
3,0z5JW1r,Teletrabajo,1
4,0z5VvGv,Full-time,55


In [19]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
2084867,1112380124,6rPXRlv,0,19.0,False,True,False,False,False,False,...,Full-time,Senior / Semi-Senior,Finanzas,24.0,0.0,0.000000,False,1,19,14
6413568,1112416648,OqroPEd,0,25.0,False,True,False,False,False,False,...,Full-time,-,Legal,4.0,0.0,0.000000,False,0,21,1
1276788,1112463121,GNJaEvp,0,32.0,False,True,False,False,False,False,...,Part-time,Otro,Marketing,8.0,0.0,0.000000,False,0,0,0
526893,1112305232,Dr6ZkrX,1,24.0,False,True,False,False,False,False,...,Full-time,Junior,Recepcionista,6.0,0.0,0.000000,False,1,52,13
3141735,1112316971,qe22eaN,1,38.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Comercial,273.0,0.0,0.000000,False,39,110,78
2056275,1112396755,JBxjz03,1,20.0,False,True,False,False,False,False,...,Full-time,Senior / Semi-Senior,Cadetería,5.0,0.0,0.000000,False,7,31,19
7002613,1112420181,lDL1ZVz,1,39.0,False,True,False,False,False,False,...,Full-time,Senior / Semi-Senior,Comercial,52.0,2.0,0.038462,True,4,33,25
1807435,1112033906,X9l1NML,1,35.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Ventas,12.0,0.0,0.000000,False,3,33,29
15255000,1112249820,NzjxdDz,0,23.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Ventas,20.0,0.0,0.000000,False,4,18,16
12247869,1112272149,vVkJQ6P,0,25.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Programación,146.0,0.0,0.000000,False,0,20,16


In [20]:
train, test=train_test_split(postulaciones.sample(2*(len(postulaciones)//3)),test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  10187849 Test:  536203


In [21]:
features=['edad','sexo_FEM', 'sexo_MASC','Doctorado_Abandonado',
'Doctorado_En Curso', 'Doctorado_Graduado', 'Master_Abandonado',
'Master_En Curso', 'Master_Graduado', 'Posgrado_Abandonado',
'Posgrado_En Curso', 'Posgrado_Graduado', 'Secundario_Abandonado',
'Secundario_En Curso', 'Secundario_Graduado', 'Terciario/Técnico_Abandonado',
'Terciario/Técnico_En Curso', 'Terciario/Técnico_Graduado', 'Universitario_Abandonado',
'Universitario_En Curso', 'Universitario_Graduado', 'pide_hombre',
'pide_mujer', 'ingles', 'experiencia', 'paquete_office', 'liderazgo', 'secundario',
'viajar', 'empresa_importante', 'capacitacion', 'remuneracion_pretendida', 'graduados',
'estudiantes', 'conocimientos_especificos',
'tipo_de_trabajo_Fines de Semana', 'vistas_postulante',
'vistas_al_aviso', 'cantidad', 'postulaciones_misma_area', 'postulaciones_mismo_tipo', 
'postulaciones_mismo_nivel', 'lo_vio']

In [22]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [23]:
postulaciones=0
c=0
gc.collect()

198

In [24]:
c=MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto',epsilon=1e-08, hidden_layer_sizes=(len(features), 5), learning_rate='constant',learning_rate_init=0.001, max_iter=200,nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=True,warm_start=False)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl")
dt=c.fit(x_train,y_train)

Iteration 1, loss = 0.32528994
Iteration 2, loss = 0.30621477
Iteration 3, loss = 0.30176977
Iteration 4, loss = 0.29943174
Iteration 5, loss = 0.29909140
Iteration 6, loss = 0.29799293
Iteration 7, loss = 0.29714785
Iteration 8, loss = 0.29611534
Iteration 9, loss = 0.29570067
Iteration 10, loss = 0.29484779
Iteration 11, loss = 0.29426416
Iteration 12, loss = 0.29408455
Iteration 13, loss = 0.29346154
Iteration 14, loss = 0.29256881
Iteration 15, loss = 0.29220574
Iteration 16, loss = 0.29202463
Iteration 17, loss = 0.29198377
Iteration 18, loss = 0.29163120
Iteration 19, loss = 0.29112570
Iteration 20, loss = 0.29066716
Iteration 21, loss = 0.29075826
Iteration 22, loss = 0.29047720
Iteration 23, loss = 0.29073396
Iteration 24, loss = 0.29016576
Iteration 25, loss = 0.28969757
Iteration 26, loss = 0.28992275
Iteration 27, loss = 0.29044403
Iteration 28, loss = 0.28984882
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [25]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9437847241860403


In [26]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/NN v4.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/NN v4.pkl']

In [29]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
40681,35314,1112410201,609WmO,28.0,True,False,False,False,False,False,...,Part-time,Senior / Semi-Senior,Comercial,8.0,0.0,0.000000,False,1.0,0.0,17.0
52427,85423,1112466636,mzje0l9,20.0,False,True,False,False,False,False,...,Full-time,Junior,Comercial,68.0,1.0,0.014706,True,12.0,64.0,17.0
42586,44348,1112426116,1QqG3ER,33.0,False,True,False,False,False,False,...,Full-time,Otro,Distribución,0.0,0.0,0.000000,False,0.0,19.0,0.0
97373,97112,1112497547,5lQx9G,45.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Programación,17.0,0.0,0.000000,False,2.0,19.0,12.0
5210,64642,1112458432,8XmbQj,28.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Selección,159.0,2.0,0.012579,True,21.0,102.0,72.0
26356,37749,1112414415,LNwdV1P,25.0,True,False,False,False,False,False,...,Full-time,Junior,Recepcionista,14.0,0.0,0.000000,False,1.0,21.0,7.0
92743,91824,1112473201,1ZRAwb,40.0,True,False,False,False,False,False,...,Full-time,Senior / Semi-Senior,Ingeniería Civil,6.0,0.0,0.000000,False,0.0,25.0,21.0
90524,58501,1112452739,EPXQBo,35.0,False,True,False,False,False,False,...,Full-time,Senior / Semi-Senior,Ventas,2.0,0.0,0.000000,False,5.0,17.0,13.0
14555,63554,1112457497,81boVL,27.0,True,False,False,False,False,False,...,Full-time,Junior,Administración,368.0,2.0,0.005435,True,13.0,115.0,31.0
12708,18374,1112213148,akjK4e5,19.0,False,True,False,False,False,False,...,Full-time,-,Almacén / Depósito / Expedición,145.0,2.0,0.013793,True,14.0,133.0,2.0


In [30]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.89257571, 0.10742429],
       [0.25623791, 0.74376209],
       [0.83756812, 0.16243188],
       ...,
       [0.96743863, 0.03256137],
       [0.93456374, 0.06543626],
       [0.98181073, 0.01818927]])

In [31]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [32]:
prediccion['sepostulo']=y_final2

In [33]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,Jefe / Supervisor / Responsable,Comercial,33.0,0.0,0.0,False,1.0,19.0,1.0,0.107424
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,Jefe / Supervisor / Responsable,Comercial,8.0,0.0,0.0,False,6.0,63.0,0.0,0.743762
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,Jefe / Supervisor / Responsable,Comercial,12.0,0.0,0.0,False,4.0,16.0,1.0,0.162432
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,Otro,Tecnologia / Sistemas,0.0,0.0,0.0,False,2.0,20.0,0.0,0.121970
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,Otro,Tecnologia / Sistemas,3.0,0.0,0.0,False,1.0,17.0,2.0,0.031626


In [34]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.107424
1,1,0.743762
2,2,0.162432
3,3,0.121970
4,4,0.031626


In [35]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion50.csv",index=False)

In [36]:
prediccion.sepostulo.mean()

0.4948427172151803